In [1]:
# DS2002 Data Project 1
# Teresa Duong
# 10/20/2024

# Modules Used:
import csv
import json
import pandas as pd
import sqlite3
from pathlib import Path

#Note: I have run through my ETL processor multiple times through each combination of input and output in order to display the results of them all
#in the cell outputs here. However, if the ETL processor is run only once, only the results related to the specified input and output will
#run. Thank you!

#To use this processor, please run each cell. Input directions will be printed, additional notes on the program will be in comments, and 
#rubric criteria will be quoted in comments. Thank you.


In [172]:
print ("Which data source would you like to process first? Please type 'CSV' for the Charlottesville Transit CSV dataset or 'JSON' for the Nobel Peace Prize Winners JSON dataset.")
input_data_type = input()


Which data source would you like to process first? Please type 'CSV' for the Charlottesville Transit CSV dataset or 'JSON' for the Nobel Peace Prize Winners JSON dataset.


 JSON


In [182]:
print("Please name the output format you would like your data source in (For CSV input, there are JSON and SQL options, for JSON input there are CSV and SQL options)")
output_data_type = input()

Please name the output format you would like your data source in (For CSV input, there are JSON and SQL options, for JSON input there are CSV and SQL options)


 SQL


In [128]:
# File Confirmation: A printed message will confirm what file we are processing.
if input_data_type == 'CSV':
    filename = "DataSource_1_Transit_2020.csv"
    print(f' Confirmation: The file we will process is {filename}')    
elif input_data_type == 'JSON':
    filename = "DataSource_2_Nobel_Peace_Prize_Winners.json"
    print(f' Confirmation: The file we will process is {filename}')
else:
    print("Error: Invalid input file format. Please type either 'CSV' or 'JSON'. Thank you.")

# Both data sources are local files in the folder. 


 Confirmation: The file we will process is DataSource_2_Nobel_Peace_Prize_Winners.json


In [148]:
# "Fetch / download / retrieve a remote data file by URL, or ingest a local file mounted." 
# (Both data sources are local files in the submitted folder)


# "In your code, Generate a brief summary of the data file ingestion including:
# 1. Number of records
# 2. Number of columns"

if input_data_type == 'CSV':
    records = []
    columns = []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        columns = next(csvreader)
        for record in csvreader:
            records.append(record)
        print("Number of Records: %d" % (len(records)))
        print("Number of Columns: %d" % len(columns))
    print("Preview of First 3 Records:")
    display(pd.read_csv(filename).head(3))

#I created lists containing the records and column names. I counted the numbers of each by finding the length of these lists.
        
elif input_data_type == 'JSON':
    with open(filename, 'r') as jsonfile:
        data = json.load(jsonfile)
        flattened_data = []     #My json file was nested, so I flattened it by adding the larger nest keys to each laureate object individually.   
        for prize in data['prizes']:
            year = prize['year']
            category = prize['category']            
            for laureate in prize.get('laureates', []):
                laureate['year'] = year
                laureate['category'] = category
                flattened_data.append(laureate) 
        print("Number of Records: %d" % len(flattened_data))
        print("Number of Columns: %d" % len(flattened_data[0].keys())) 
        print("Preview of First 3 Objects:")
        print(json.dumps(flattened_data[:3], indent=4))

#I flattened my json file in order to count the record and columns, and also to prepare it for conversion to CSV and SQL table formats later on.
#I counted the number of records by finding the length/number of objects in my flattened data structure.
#I counted the number of columns by finding the length/number of keys in the first object, since all objects in this dataset have the same keys.

Number of Records: 1012
Number of Columns: 7
Preview of First 3 Objects:
[
    {
        "id": "1039",
        "firstname": "David",
        "surname": "Baker",
        "motivation": "\"for computational protein design\"",
        "share": "2",
        "year": "2024",
        "category": "chemistry"
    },
    {
        "id": "1040",
        "firstname": "Demis",
        "surname": "Hassabis",
        "motivation": "\"for protein structure prediction\"",
        "share": "4",
        "year": "2024",
        "category": "chemistry"
    },
    {
        "id": "1041",
        "firstname": "John",
        "surname": "Jumper",
        "motivation": "\"for protein structure prediction\"",
        "share": "4",
        "year": "2024",
        "category": "chemistry"
    }
]


In [150]:
# "Modify the number of columns from the source to the destination, reducing or adding columns. If you add data cols you can put any other
# useful information in that column you wish"

if input_data_type == 'CSV':
    with open(filename, 'r') as csvfile, open('modified_transit.csv', 'w') as modified_csvfile:
        csvreader = csv.reader(csvfile)
        writer = csv.writer(modified_csvfile)
        columns = next(csvreader)
        remove_column = 'Fare'
#The Charlottesville Transit system has discontinued the use of fares to admit passengers since 2020, so I chose to remove this column since it is 
#less relevant today.
        column_to_remove_index = columns.index(remove_column)
        modified_columns = [col for i, col in enumerate(columns) if i != column_to_remove_index]
        writer.writerow(modified_columns)
        modified_records = []  
        for record in csvreader:
            modified_record = [value for i, value in enumerate(record) if i != column_to_remove_index]
            writer.writerow(modified_record)  # Write the modified row to the new CSV file
            modified_records.append(record)
# "In your code Generate a brief summary of the post processing including:
#1. Number of records
#2. Number of columns"
        print("Number of records in modified CSV: %d" % len(modified_records))
        print("Number of columns in modified CSV: %d" % len(modified_columns))
        print("Preview of first 3 records:")
        display(pd.read_csv('modified_transit.csv').head(3))

elif input_data_type == 'JSON':
    with open('modified_nobel.json', 'w') as modified_jsonfile:
        modified_flattened_data = []
        for laureate in flattened_data:
            laureate.pop('motivation', None) # I removed the 'motivation' column since the column values had a string of text datatype which could allow for more variance, less normalization.
            modified_flattened_data.append(laureate)
        json.dump(flattened_data, modified_jsonfile, indent=4)
# "In your code Generate a brief summary of the post processing including:
#1. Number of records
#2. Number of columns"
        print("Number of records in modified JSON: %d" % len(flattened_data))
        print("Number of columns in modified JSON: %d" % len(flattened_data[0].keys()))
        print("Preview of first 3 objects:")
        print(json.dumps(flattened_data[:3], indent=4))
else:
    print("Error: Invalid output file format. Please type either 'CSV', 'JSON', or 'SQL'. Thank you.")

#To remove a column from the Transit CSV dataset, I wrote to a new CSV only rows and columns that do NOT have column = 'Fare'.
#To remove a column from the Nobel Peace Prize Winners JSON dataset, used the pop() method to remove the column 'motivation' and appended the modified
#objects to modified_flattened_data. I dumped this data into a new JSON file.
#I used similar methods as in the previous cell to count the number of records and columns.

Number of records in modified JSON: 1012
Number of columns in modified JSON: 6
Preview of first 3 objects:
[
    {
        "id": "1039",
        "firstname": "David",
        "surname": "Baker",
        "share": "2",
        "year": "2024",
        "category": "chemistry"
    },
    {
        "id": "1040",
        "firstname": "Demis",
        "surname": "Hassabis",
        "share": "4",
        "year": "2024",
        "category": "chemistry"
    },
    {
        "id": "1041",
        "firstname": "John",
        "surname": "Jumper",
        "share": "4",
        "year": "2024",
        "category": "chemistry"
    }
]


In [162]:
# "Convert the general format and data structure of the data source (from JSON to CSV, from CSV to JSON, from JSON into a SQL database table, etc.)"
# "I want the option to convert any source to any target. So, if I get a CSV as an input, I want the user to choose an output"

#Convert CSV to Dataframe Initially
if input_data_type == 'CSV':
    transit_df = pd.read_csv("modified_transit.csv")
    print("Preview of first 3 records:")
    display(transit_df.head(2))

#This cell will run if the user has chosen a CSV file to process. 
#I chose to convert the CSV to a panda dataframe first so that it can be used in both JSON and SQL table conversions later.

Preview of first 3 records:


,TransitID,Stop,Route,Date_Time,Count,FareCategory,PaymentType,Latitude,Longitude
0,2453,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 17:09:00+00,1,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078
1,2454,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 17:09:00+00,1,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078


In [93]:
#Convert Dataframe --> JSON
if input_data_type == 'CSV' and output_data_type == 'JSON':
    with open('transit_json_conversion.json', 'w') as jsonfile:
        transit_json = transit_df.to_json(orient='records', indent=4)
        transit_deserialized = json.loads(transit_json)
        json.dump(transit_deserialized, jsonfile, indent=4)
        print ("Preview of First 3 Objects of Modified Transit JSON file:")
        print(json.dumps(transit_deserialized[:3], indent=4))
#If the user has specified both an input of CSV and an output of JSON, this cell will run.
#To covert the dataframe we made in the previous cell, I used the .to_json method.
#In order to display a previw of the first 3 objects, I converted the JSON to Python and called the first three objects.
#To store the converted new file, I wrote the JSON data structure to transit_json_conversion.json.

Preview of First 3 Objects of Modified Transit JSON file:
[
    {
        "TransitID": 2453,
        "Stop": "10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location",
        "Route": "07 A",
        "Date_Time": "2020/01/18 17:09:00+00",
        "Count": 1,
        "FareCategory": "UVA Academic",
        "PaymentType": "Barcode-3rd Party Media",
        "Latitude": 38.0363,
        "Longitude": -78.5078
    },
    {
        "TransitID": 2454,
        "Stop": "10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location",
        "Route": "07 A",
        "Date_Time": "2020/01/18 17:09:00+00",
        "Count": 1,
        "FareCategory": "UVA Academic",
        "PaymentType": "Barcode-3rd Party Media",
        "Latitude": 38.0363,
        "Longitude": -78.5078
    },
    {
        "TransitID": 2455,
        "Stop": "10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location",
        "Route": "07 A",

In [170]:
# Convert Dataframe --> SQL Table
if input_data_type == 'CSV' and output_data_type == 'SQL':
    Path("transit_sql_conversion.db").touch()
    connection =sqlite3.connect("transit_sql_conversion.db")
    cursor = connection.cursor()
    
    transit_df.to_sql('transit_table', connection, if_exists='replace', index=False)

    #Preview of First 3 Rows
    print("Preview of First 3 Rows of Modified Transit SQL Table:")
    cursor.execute("SELECT * FROM transit_table LIMIT 3")
    column_names = [description[0] for description in cursor.description]
    print("Header:", column_names)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    
    connection.close()
#I first created a new database "transit_sql_conversion.db" and connected to it.
#I then used the to_sql() method to convert the dataframe we made two cells above into a SQL table.
#To preview the first 3 rows of the table, I used SELECT and printed each row I selected as well as the column names (Header). 

Preview of First 3 Rows of Modified Transit SQL Table:
Header: ['TransitID', 'Stop', 'Route', 'Date_Time', 'Count', 'FareCategory', 'PaymentType', 'Latitude', 'Longitude']
(2453, '10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location', '07 A', '2020/01/18 17:09:00+00', 1, 'UVA Academic', 'Barcode-3rd Party Media', 38.0363, -78.5078)
(2454, '10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location', '07 A', '2020/01/18 17:09:00+00', 1, 'UVA Academic', 'Barcode-3rd Party Media', 38.0363, -78.5078)
(2455, '10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location', '07 A', '2020/01/18 17:09:00+00', 1, 'UVA Academic', 'Barcode-3rd Party Media', 38.0363, -78.5078)


In [138]:
# Convert JSON --> Dataframe Initially
if input_data_type == 'JSON':
    with open('modified_nobel.json', 'r') as jsonfile:
        nobel_data = json.load(jsonfile)
        nobel_df = pd.DataFrame(nobel_data)
        print("Preview of first 3 records:")
        display(nobel_df.head(3))
#Similarly, for user requests to process a JSON file, I initially converted the JSON to a panda dataframe to prepare it for CSV and SQL table conversions.

Preview of first 3 records:


,id,firstname,surname,share,year,category
0,1039,David,Baker,2,2024,chemistry
1,1040,Demis,Hassabis,4,2024,chemistry
2,1041,John,Jumper,4,2024,chemistry


In [180]:
# Convert Dataframe --> CSV
if input_data_type == 'JSON' and output_data_type == 'CSV':
    nobel_df.to_csv('nobel_csv_conversion.csv', index=False)
    print("Preview of first 3 records:")
    display(pd.read_csv('nobel_csv_conversion.csv').head(3))
#I used the .to_csv() method to convert the dataframe we made in the previous cell into a CSV file.

Preview of first 3 records:


,id,firstname,surname,share,year,category
0,1039,David,Baker,2,2024,chemistry
1,1040,Demis,Hassabis,4,2024,chemistry
2,1041,John,Jumper,4,2024,chemistry


In [184]:
# Convert Dataframe --> SQL
if input_data_type == 'JSON' and output_data_type == 'SQL':
    Path("nobel_sql_conversion.db").touch()
    connection =sqlite3.connect("nobel_sql_conversion.db")
    cursor = connection.cursor()
    
    nobel_df.to_sql('nobel_table', connection, if_exists='replace', index=False)

    # Preview of First 5 Rows in SQL Table:
    print("A preview of the first 3 rows:")
    cursor.execute("SELECT * FROM nobel_table LIMIT 3")
    column_names = [description[0] for description in cursor.description]
    print("Header:", column_names)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    
    connection.close()
#Similarly to our CSV --> SQL conversion, I first created a new database "nobel_sql_conversion" and connected to it.
#I then created a table within the database, "nobel_table"
#To preview the first 3 rows of the table, I used SELECT and printed each row I selected as well as the column names (Header). 

A preview of the first 3 rows:
Header: ['id', 'firstname', 'surname', 'share', 'year', 'category']
('1039', 'David', 'Baker', '2', '2024', 'chemistry')
('1040', 'Demis', 'Hassabis', '4', '2024', 'chemistry')
('1041', 'John', 'Jumper', '4', '2024', 'chemistry')


In [158]:
# "Store The converted (new) file should be written to disk (local file) or written to a SQL database."
if input_data_type == 'CSV' and output_data_type == 'JSON':
    print("The path to the new stored JSON file:")
    transit_json_path = Path('transit_json_conversion.json').resolve()
    print(transit_json_path)
elif input_data_type == 'CSV' and output_data_type == 'SQL':
    print("The path to the new stored SQL database:")
    transit_db_path = Path("transit_sql_conversion.db").resolve()
    print(transit_db_path)
elif input_data_type == 'JSON' and output_data_type == 'CSV':
    print("The path to the new stored CSV file:")
    transit_db_path = Path('nobel_csv_conversion.csv').resolve()
    print(transit_db_path)
elif input_data_type == 'JSON' and output_data_type == 'SQL':
    print("The path to the new stored SQL database:")
    transit_db_path = Path("nobel_sql_conversion.db").resolve()
    print(transit_db_path)

#Here I provided the paths to all created files to show that I have I locally stored our converted (new) files.

The path to the new stored SQL database:
C:\Users\13012\OneDrive - University of Virginia\3rd Year 24-25\Data Science Systems\Project 1\nobel_sql_conversion.db


In [190]:
print("Thank you for using this processor! Please clear outputs and return to cell 1 to process another file or receive a different format output. Have a wonderful day!")

Thank you for using this processor! Please clear outputs and return to cell 1 to process another file or receive a different format output. Have a wonderful day!
